INSPER - Big Data e Computação em Nuvem

# Projeto Final

- Guilherme Fregulia
- Rodrigo França
- Tamara Alves
- Vinicius Vargas

In [2]:
!pip install -q holidays

In [4]:
! pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.1 MB/s eta 0:00:00


## Importando Dados

No código abaixo criamos a sessão SparK e fazemos a importação e tratamento inicial do nosso conjunto de dados.

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import pyspark.sql.functions as f
from pyspark.sql.types import StringType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from functools import reduce
from pyspark.sql.types import *

from pyspark.sql.functions import *

from datetime import date, datetime
import datetime
import holidays
import pandas as pd

from datetime import date
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [8]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[4]") \
            .appName("nyc_pisloteros_de_los_datos>") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,com.microsoft.azure:azure-storage:8.6.6") \
            .getOrCreate()

In [7]:
STORAGE_ACCOUNT = 'dlspadseastusprod'
CONTAINER = 'big-data-comp-nuvem'
FOLDER = 'airline-delay'
TOKEN = 'lSuH4ZI9BhOFEhCF/7ZQbrpPBIhgtLcPDfXjJ8lMxQZjaADW4p6tcmiZGDX9u05o7FqSE2t9d2RD+ASt0YFG8g=='

spark.conf.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)

In [9]:
config = spark.sparkContext._jsc.hadoopConfiguration()
config.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)
sc = spark.sparkContext

In [10]:
labels =(
        ('FL_DATE', DateType()),
        ('OP_CARRIER', StringType()),
        ('OP_CARRIER_FL_NUM', IntegerType()),
        ('ORIGIN', StringType()),
        ('DEST', StringType()),
        ('CRS_DEP_TIME', FloatType()),
        ('DEP_TIME', FloatType()),
        ('DEP_DELAY', FloatType()),
        ('TAXI_OUT', FloatType()),
        ('WHEELS_OFF', FloatType()),
        ('WHEELS_ON', FloatType()),
        ('TAXI_IN', FloatType()),
        ('CRS_ARR_TIME', FloatType()),
        ('ARR_TIME', FloatType()),
        ('ARR_DELAY', FloatType()),
        ('CANCELLED', FloatType()),
        ('CANCELLATION_CODE', StringType()),
        ('DIVERTED', FloatType()),
        ('CRS_ELAPSED_TIME', FloatType()),
        ('ACTUAL_ELAPSED_TIME', FloatType()),
        ('AIR_TIME', FloatType()),
        ('DISTANCE', FloatType()),
        ('CARRIER_DELAY', FloatType()),
        ('WEATHER_DELAY', FloatType()),
        ('NAS_DELAY', FloatType()),
        ('SECURITY_DELAY', FloatType()),
        ('LATE_AIRCRAFT_DELAY', FloatType())
)

my_schema = StructType([StructField(x[0], x[1], True) for x in labels])

In [11]:
### ALTERAR SCRIPT PARA LEITURA - EMBELEZAR - VINI
df09 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2009.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df10 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2010.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df11 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2011.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df12 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2012.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df13 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2013.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df14 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2014.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df15 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2015.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df16 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2016.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df17 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2017.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

df18 = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2018.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema = my_schema)

In [12]:
# criando uma lista com todos os nossos data frames
dfs = [df09, df10, df11, df12, df13, df14, df15, df16, df17, df18]

# Unindo os data frames
merged_df = reduce(lambda x, y: x.union(y), dfs)

## Análise Exploratória

É esperado que dias com muitos voos ocasionem mais atrasos. Por sua vez, espera-se que em datas festivas, finais de semanas e feriados o trânsito em aeroportos seja substancialmente mais alto.

Abaixo, fazemos uma análise exploratória dos dados a fim de averiguar esta hipótese.

In [13]:
us_holidays = holidays.US()
feriados = holidays.US([2009,2010,2011,2012,2013,2014,2015,2016,2017,2018])
dct = {k:[v] for k,v in feriados.items()}
feriados_US = pd.DataFrame(dct)
feriados_US = feriados_US.T
feriados_US  = feriados_US.rename_axis('Data').reset_index()
feriados_US = feriados_US['Data']
dias_anteriores = []

for i in feriados_US:
    dias_anteriores.append(i - datetime.timedelta(days=1))

feriados_US  = feriados_US.to_list()
dias_feriados = dias_anteriores + feriados_US
dias_feriados.sort()

In [14]:
merged_df = merged_df.withColumn("is_holiday", f.when(f.col("FL_DATE").isin(dias_feriados),1).otherwise(0))

df10_adj = merged_df.filter(year(merged_df.FL_DATE) == 2010)

merged_df.cache()
df10_adj.cache()

DataFrame[FL_DATE: date, OP_CARRIER: string, OP_CARRIER_FL_NUM: int, ORIGIN: string, DEST: string, CRS_DEP_TIME: float, DEP_TIME: float, DEP_DELAY: float, TAXI_OUT: float, WHEELS_OFF: float, WHEELS_ON: float, TAXI_IN: float, CRS_ARR_TIME: float, ARR_TIME: float, ARR_DELAY: float, CANCELLED: float, CANCELLATION_CODE: string, DIVERTED: float, CRS_ELAPSED_TIME: float, ACTUAL_ELAPSED_TIME: float, AIR_TIME: float, DISTANCE: float, CARRIER_DELAY: float, WEATHER_DELAY: float, NAS_DELAY: float, SECURITY_DELAY: float, LATE_AIRCRAFT_DELAY: float, is_holiday: int]

In [15]:
merged_df.limit(5).toPandas().fillna(0)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,is_holiday
0,2009-01-01,XE,1204,DCA,EWR,1100.0,1058.0,-2.0,18.0,1116.0,...,62.0,68.0,42.0,199.0,0.0,0.0,0.0,0.0,0.0,1
1,2009-01-01,XE,1206,EWR,IAD,1510.0,1509.0,-1.0,28.0,1537.0,...,82.0,75.0,43.0,213.0,0.0,0.0,0.0,0.0,0.0,1
2,2009-01-01,XE,1207,EWR,DCA,1100.0,1059.0,-1.0,20.0,1119.0,...,70.0,62.0,36.0,199.0,0.0,0.0,0.0,0.0,0.0,1
3,2009-01-01,XE,1208,DCA,EWR,1240.0,1249.0,9.0,10.0,1259.0,...,77.0,56.0,37.0,199.0,0.0,0.0,0.0,0.0,0.0,1
4,2009-01-01,XE,1209,IAD,EWR,1715.0,1705.0,-10.0,24.0,1729.0,...,105.0,77.0,40.0,213.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
# Top 10 companhias aereas com mais voos entre 2009 e 2010
top_10_airlines = merged_df.groupBy('OP_CARRIER').count()\
    .orderBy(desc('count'))\
    .head(10)

In [ ]:
# Group the flights by airline and count the number of flights per airline
airline_counts = merged_df.groupBy('OP_CARRIER').count()

# Sort the airlines by the number of flights in descending order
airline_counts = airline_counts.orderBy(desc('count'))

# Take the top 10 most common airlines
top_10_airlines = airline_counts.head(10)

# Print the top 10 airlines by number of flights
print(top_10_airlines)

top_10_air = merged_df.groupBy('OP_CARRIER').count()

In [ ]:
airline_codes = [row[0] for row in top_10_airlines]
flight_counts = [row[1] for row in top_10_airlines]

# Plot the top 10 airlines by number of flights
plt.bar(airline_codes, flight_counts)
plt.title('Top 10 Airlines by Number of Flights')
plt.xlabel('Airline')
plt.ylabel('Number of Flights')
plt.show()

In [ ]:
airport_counts = merged_df.groupBy(['ORIGIN', 'DEST']).count()

# Sort the airlines by the number of flights in descending order
airport_counts = airport_counts.orderBy(desc('count'))

# Take the top 10 most common airlines
top_10_airports = airport_counts.head(10)

# Print the top 10 airlines by number of flights
print(top_10_airports)



In [ ]:
airport_codes = [row[0:1] for row in top_10_airports]
flight_num = [row[2] for row in top_10_airports]

x = [f"{d[0]} to {d[1]}" for d in top_10_airports]
y = [d[2] for d in top_10_airports]

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x, y)
ax.set_xlabel('Route')
ax.set_ylabel('Count')
ax.set_title('Rotas mais Comuns')
plt.xticks(rotation=45, ha='right')
plt.show()

## Pre-processamento dados

Nesta seção fazemos o pre-processamento dos dados antes de treinarmos os modelos de predição.

In [16]:
#model_df = merged_df
model_df = df10_adj

In [36]:
model_df1 = (
    model_df
    .withColumn('ATRASO', when(model_df.ARR_DELAY > 0, 1).otherwise(0))
    .filter(model_df.CANCELLED == 0)
    .drop(
        'ARR_TIME',
        'ARR_DELAY',
        'WHEELS_ON',
        'FL_DATE',
        'OP_CARRIER_FL_NUM',
        'CANCELLED',
        'CANCELLATION_CODE',
        'CARRIER_DELAY',
        'WEATHER_DELAY',
        'NAS_DELAY', 'SECURITY_DELAY', 'TAXI_IN', 'LATE_AIRCRAFT_DELAY')
    .fillna(0)
)

### ADICIONAR MES OU ÉPOCA DO ANO

In [37]:
model_df1.printSchema()

root
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: float (nullable = false)
 |-- DEP_TIME: float (nullable = false)
 |-- DEP_DELAY: float (nullable = false)
 |-- TAXI_OUT: float (nullable = false)
 |-- WHEELS_OFF: float (nullable = false)
 |-- CRS_ARR_TIME: float (nullable = false)
 |-- DIVERTED: float (nullable = false)
 |-- CRS_ELAPSED_TIME: float (nullable = false)
 |-- ACTUAL_ELAPSED_TIME: float (nullable = false)
 |-- AIR_TIME: float (nullable = false)
 |-- DISTANCE: float (nullable = false)
 |-- is_holiday: integer (nullable = false)
 |-- ATRASO: integer (nullable = false)



Separando os dados entre treino e teste

In [19]:
train_df, test_df = model_df1.randomSplit([0.8, 0.2], seed=42)
toy_df = test_df.sample(False, 0.01, seed=42)

In [ ]:
print("Train set count: ", train_df.count())
print("Test set count:", test_df.count())
print("Toy df count:", toy_df.count())

In [35]:
train_df.limit(10).toPandas()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,is_holiday,ATRASO
0,9E,ABE,DTW,600.0,0.0,0.0,0.0,0.0,0.0,0.0,746.0,1.0,A,0.0,106.0,0.0,0.0,424.0,0,0
1,9E,ABE,DTW,600.0,0.0,0.0,0.0,0.0,0.0,0.0,746.0,1.0,C,0.0,106.0,0.0,0.0,424.0,0,0
2,9E,ABE,DTW,600.0,550.0,-10.0,37.0,627.0,802.0,4.0,746.0,0.0,None,0.0,106.0,136.0,95.0,424.0,0,1
3,9E,ABE,DTW,600.0,551.0,-9.0,13.0,604.0,727.0,8.0,746.0,0.0,None,0.0,106.0,104.0,83.0,424.0,0,0
4,9E,ABE,DTW,600.0,552.0,-8.0,20.0,612.0,733.0,10.0,746.0,0.0,None,0.0,106.0,111.0,81.0,424.0,0,0
5,9E,ABE,DTW,600.0,552.0,-8.0,20.0,612.0,742.0,14.0,746.0,0.0,None,0.0,106.0,124.0,90.0,424.0,0,1
6,9E,ABE,DTW,600.0,554.0,-6.0,29.0,623.0,741.0,10.0,746.0,0.0,None,0.0,106.0,117.0,78.0,424.0,0,1
7,9E,ABE,DTW,600.0,555.0,-5.0,12.0,607.0,733.0,15.0,746.0,0.0,None,0.0,106.0,113.0,86.0,424.0,0,1
8,9E,ABE,DTW,600.0,555.0,-5.0,14.0,609.0,724.0,17.0,746.0,0.0,None,0.0,106.0,106.0,75.0,424.0,0,0
9,9E,ABE,DTW,600.0,555.0,-5.0,20.0,615.0,728.0,8.0,746.0,0.0,None,0.0,106.0,101.0,73.0,424.0,0,0


Tratando variáveis categóricas com One Hot Enconder

In [21]:
cat_feats = [
    'OP_CARRIER',
    'ORIGIN',
    'DEST',
    'CANCELLATION_CODE'
]

indexOutputCols = [x + "Index" for x in cat_feats]
oheOutputCols = [x + "OHE" for x in cat_feats]

stringIndexer = StringIndexer(inputCols = cat_feats,
                              outputCols = indexOutputCols,
                              handleInvalid = "skip")

oheEncoder = OneHotEncoder(inputCols = indexOutputCols, outputCols = oheOutputCols )

Tratando variáveis numéricas

In [22]:
num_feats = [
    'CRS_DEP_TIME',
    'DEP_TIME',
    'DEP_DELAY',
    'TAXI_OUT',
    'WHEELS_OFF',
    'WHEELS_ON',
    'TAXI_IN',
    'CRS_ARR_TIME',
    'CANCELLED',
    'DIVERTED',
    'CRS_ELAPSED_TIME',
    'ACTUAL_ELAPSED_TIME',
    'AIR_TIME',
    'DISTANCE',
    'is_holiday'
]

numVecAssembler = VectorAssembler(inputCols = num_feats, outputCol = "features", handleInvalid='skip')

stdScaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [23]:
assemblerInputs = oheOutputCols + ['features_scaled']

vecAssembler = VectorAssembler(inputCols = assemblerInputs, outputCol = "features_vector")

In [24]:
stages = [stringIndexer, oheEncoder, numVecAssembler, stdScaler, vecAssembler]

In [25]:
transform_pipeline =  Pipeline(stages=stages)
fitted_transformer = transform_pipeline.fit(train_df)
transformed_train_df = fitted_transformer.transform(train_df)

transformed_train_df.limit(10).toPandas()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,ORIGINIndex,DESTIndex,CANCELLATION_CODEIndex,OP_CARRIEROHE,ORIGINOHE,DESTOHE,CANCELLATION_CODEOHE,features,features_scaled,features_vector
0,9E,ABE,DTW,600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0)","(600.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 746.0, 1...","(1.299072822445428, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,9E,ABE,DTW,600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0)","(600.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 746.0, 1...","(1.299072822445428, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,9E,ABE,DTW,610.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0)","(610.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 805.0, 1...","(1.3207240361528516, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,9E,ABE,DTW,610.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(1.0, 0.0, 0.0)","(610.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 805.0, 1...","(1.3207240361528516, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,9E,ABE,DTW,610.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0)","(610.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 805.0, 1...","(1.3207240361528516, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,9E,ABE,DTW,610.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0)","(610.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 805.0, 1...","(1.3207240361528516, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,9E,ABE,DTW,1230.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0)","(1230.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1424.0,...","(2.6630992860131273, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,9E,ABE,DTW,1235.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(1.0, 0.0, 0.0)","(1235.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1424.0,...","(2.673924892866839, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,9E,ABE,DTW,1525.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151.0,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0)","(1525.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1715.0,...","(3.301810090382129, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,9E,AEX,MEM,1120.0,0.0,0.0,0.0,0.0,0.0,0.0,...,161.0,24.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(11

In [26]:
train_df.cache()

DataFrame[OP_CARRIER: string, ORIGIN: string, DEST: string, CRS_DEP_TIME: float, DEP_TIME: float, DEP_DELAY: float, TAXI_OUT: float, WHEELS_OFF: float, WHEELS_ON: float, TAXI_IN: float, CRS_ARR_TIME: float, CANCELLED: float, CANCELLATION_CODE: string, DIVERTED: float, CRS_ELAPSED_TIME: float, ACTUAL_ELAPSED_TIME: float, AIR_TIME: float, DISTANCE: float, is_holiday: int, ATRASO: int]

## Modelagem

### Árvore de Decisão

In [27]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier


In [28]:
# Train a DecisionTree model.
labelCol = 'ATRASO'

dt = DecisionTreeClassifier(labelCol=labelCol, \
                            featuresCol="features_vector", \
                            impurity="gini")

pipe_stages = stages + [dt]

pipe_dt = Pipeline(stages=pipe_stages)

In [29]:
fitted_pipe = pipe_dt.fit(train_df)

In [30]:
preds = fitted_pipe.transform(test_df)

In [31]:
preds.limit(10).toPandas()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,OP_CARRIEROHE,ORIGINOHE,DESTOHE,CANCELLATION_CODEOHE,features,features_scaled,features_vector,rawPrediction,probability,prediction
0,9E,ABE,DTW,1525.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0)","(1525.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1715.0,...","(3.301810090382129, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
1,9E,ALB,DTW,1920.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0)","(1920.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2125.0,...","(4.157033031825369, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
2,9E,ATL,EWR,1215.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1433.0,...","(2.6306224654519914, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
3,9E,ATL,EWR,1215.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1433.0,...","(2.6306224654519914, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
4,9E,ATL,EWR,1215.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1433.0,...","(2.6306224654519914, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
5,9E,ATL,HOU,1350.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1506.0,...","(2.922913850502213, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
6,9E,ATL,HOU,1350.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1506.0,...","(2.922913850502213, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
7,9E,ATL,HOU,1605.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1605.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1721.0,...","(3.47501980004152, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
8,9E,ATL,HOU,1851.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0)","(1851.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2005.0,...","(4.007639657244145, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90612.0],[1.0],0.0
9,9E,ATL,HPN,835.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator( \
                  labelCol=labelCol, \
                  predictionCol="prediction", \
                  metricName="accuracy")

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Evaluate the accuracy of the predictions
evaluator = MulticlassClassificationEvaluator(labelCol=labelCol, metricName='accuracy')
accuracy = evaluator.evaluate(preds)

# Print the accuracy
print('Accuracy of DecisionTree model:', accuracy)


Accuracy of DecisionTree model: 1.0
